In [44]:
import gym
import numpy as np

In [49]:
env = gym.make('gym_gridworld:gridworld-v2', size=3) 

In [50]:
env.render()

[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [51]:
env.observation_space

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

In [65]:
policy = [0.25, 0.25, 0.25, 0.25]

def iterativePolicyEvaluation(theta, gamma):
    S = env.observation_space
    V = [0 for s in S]
    while True:
        delta = 0
        for s, s_el in enumerate(S):
            v = V[s]
            
            i = 0
            for action, action_prob in enumerate(policy):
                tr = env.tr[env.observation_space.index(s_el)][action]
                s_prime = tr[0]
                v_prime = V[env.observation_space.index(s_prime)]
                r = tr[1]
                i += action_prob * (r + gamma * v_prime)
                print(V[env.observation_space.index(s_prime)])
            print('---------------------')
            
            V[s] = i
            delta = max(delta, abs(v - V[s]))
        
        break
        #print(delta)
        print(np.around(V, 2))
        
        if delta < theta:
            break
                    

In [66]:
iterativePolicyEvaluation(0.1, 1)

0
0
0
0
---------------------
0
0
0
-1.0
---------------------
0
0
0
-1.25
---------------------
-1.0
0
0
0
---------------------
-1.25
0
0
-1.25
---------------------
-1.3125
0
0
-1.625
---------------------
-1.25
0
0
0
---------------------
-1.625
0
0
-1.3125
---------------------
-1.484375
0
0
-1.484375
---------------------


In [9]:
env.tr[env.observation_space.index(env.world)][1][1]

-1

In [6]:
env.observation_space.index(env.world)

0

In [46]:
np.around([1.21111111,2, 1.34444444444444444], 2)

array([1.21, 2.  , 1.34])

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('gym_gridworld:gridworld-v3') 

In [10]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value. Ref: Sutton book eq. 4.6.
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        print(np.around(V, 2))
        if delta < theta:
            break
    return np.array(V)

In [11]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

[ 0.   -1.   -1.25 -1.31 -1.   -1.5  -1.69 -1.75 -1.25 -1.69 -1.84 -1.9
 -1.31 -1.75 -1.9   0.  ]
[ 0.   -1.94 -2.55 -2.73 -1.94 -2.81 -3.24 -3.4  -2.55 -3.24 -3.57 -3.22
 -2.73 -3.4  -3.22  0.  ]
[ 0.   -2.82 -3.83 -4.18 -2.82 -4.03 -4.71 -4.88 -3.83 -4.71 -4.96 -4.26
 -4.18 -4.88 -4.26  0.  ]
[ 0.   -3.67 -5.1  -5.58 -3.67 -5.19 -6.03 -6.19 -5.1  -6.03 -6.15 -5.15
 -5.58 -6.19 -5.15  0.  ]
[ 0.   -4.49 -6.3  -6.91 -4.49 -6.26 -7.22 -7.37 -6.3  -7.22 -7.19 -5.93
 -6.91 -7.37 -5.93  0.  ]
[ 0.   -5.26 -7.43 -8.16 -5.26 -7.24 -8.31 -8.44 -7.43 -8.31 -8.12 -6.62
 -8.16 -8.44 -6.62  0.  ]
[ 0.   -5.98 -8.47 -9.3  -5.98 -8.14 -9.29 -9.41 -8.47 -9.29 -8.96 -7.25
 -9.3  -9.41 -7.25  0.  ]
[  0.    -6.65  -9.43 -10.36  -6.65  -8.97 -10.19 -10.3   -9.43 -10.19
  -9.72  -7.82 -10.36 -10.3   -7.82   0.  ]
[  0.    -7.26 -10.31 -11.34  -7.26  -9.73 -11.02 -11.12 -10.31 -11.02
 -10.42  -8.34 -11.34 -11.12  -8.34   0.  ]
[  0.    -7.83 -11.12 -12.23  -7.83 -10.42 -11.77 -11.86 -11.12 -11.77
 -11.05

In [5]:
v

array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569,   0.        ])